AkHQ: http://localhost:8082  
Spark-Master http://localhost:8083  
Spark-Worker-1 http://localhost:8084

https://github.com/aehrc/pathling/tree/issue/452/lib/python#python-api-for-pathling

# Try Connect To Test FHIR Kafka

In [1]:
!python --version
!yes | pip uninstall pathling
!yes | pip install pathling==5.0.3.dev0

Python 3.10.4
Found existing installation: pathling 5.0.3.dev0
Uninstalling pathling-5.0.3.dev0:
  Would remove:
    /opt/conda/lib/python3.10/site-packages/pathling-5.0.3.dev0.dist-info/*
    /opt/conda/lib/python3.10/site-packages/pathling/*
    /opt/conda/share/pathling/examples/data/bundles/Bennett146_Swaniawski813_704c9750-f6e6-473b-ee83-fbd48e07fe3f.json
    /opt/conda/share/pathling/examples/data/bundles/Dino214_Parisian75_40d82b80-b682-cd8b-da6d-396809878641.json
    /opt/conda/share/pathling/examples/data/resources/Condition.ndjson
    /opt/conda/share/pathling/examples/data/resources/Patient.ndjson
    /opt/conda/share/pathling/examples/load_bundle.py
    /opt/conda/share/pathling/examples/load_resources.py
Proceed (Y/n)?   Successfully uninstalled pathling-5.0.3.dev0
yes: standard output: Broken pipe
  Using cached pathling-5.0.3.dev0-py2.py3-none-any.whl (19.6 MB)
yes: standard output: Broken pipe


In [2]:
import os
os.environ

environ{'SHELL': '/bin/bash',
        'HOSTNAME': 'a852bf03eb5c',
        'LANGUAGE': 'en_US.UTF-8',
        'SPARK_OPTS': '--driver-java-options=-Xms1024M --driver-java-options=-Xmx4096M --driver-java-options=-Dlog4j.logLevel=info',
        'NB_UID': '1000',
        'PWD': '/home/jovyan',
        'PYSPARK_SUBMIT_ARGS': '     --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,au.csiro.pathling:encoders:5.0.3-SNAPSHOT     --repositories https://oss.sonatype.org/content/repositories/snapshots     pyspark-shell',
        'HOME': '/home/jovyan',
        'LANG': 'en_US.UTF-8',
        'NB_GID': '100',
        'XDG_CACHE_HOME': '/home/jovyan/.cache/',
        'APACHE_SPARK_VERSION': '3.2.1',
        'PYTHONPATH': '/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip:/usr/local/spark/python:',
        'HADOOP_VERSION': '3.2',
        'SHLVL': '0',
        'CONDA_DIR': '/opt/conda',
        'SPARK_HOME': '/usr/local/spark',
        'SPARK_CONF_DIR': '/usr/local/spark/conf',
        'NB_U

In [3]:
# for debugging
#os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1 pyspark-shell"

In [4]:
appName = "Kafka, Spark and FHIR Data"
master = "spark://spark-master:7077"
#master = "local[*]"
kafka_topic = "fhir.post-gateway-kdb"

In [5]:
from pyspark.sql import SparkSession
    
spark = SparkSession.builder \
        .appName(appName) \
        .master(master) \
        .config('spark.ui.port','0') \
        .getOrCreate()

In [6]:
# https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html
# default for startingOffsets is "latest", but "earliest" allows rewind for missed alerts    
df = spark \
  .readStream  \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka1:19092") \
  .option("subscribe", kafka_topic) \
  .option("startingOffsets", "earliest") \
  .load()

In [7]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [8]:
query = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
          .writeStream \
          .queryName("gettable") \
          .format("memory") \
          .start()

# close connection after 30 seconds
query.awaitTermination(30)

False

In [9]:
kafka_data = spark.sql("select * from gettable")
kafka_data.show()
type(kafka_data)

+-------------------+--------------------+
|                key|               value|
+-------------------+--------------------+
|    Patient/patient|{"resourceType": ...|
|Condition/condition|{"resourceType": ...|
|Condition/condition|{"resourceType": ...|
|Procedure/procedure|{"resourceType": ...|
|Encounter/encounter|{"resourceType": ...|
+-------------------+--------------------+



pyspark.sql.dataframe.DataFrame

In [10]:
pd_df = kafka_data.toPandas()
pd_df

,key,value
0,Patient/patient,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
1,Condition/condition,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
2,Condition/condition,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
3,Procedure/procedure,"{""resourceType"": ""Bundle"",""meta"": {""security"":..."
4,Encounter/encounter,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."


# Bring Pathling into the game

In [12]:
from pathling.r4 import bundles

resources = bundles.from_json(kafka_data, 'value')

TypeError: 'JavaPackage' object is not callable

In [ ]:
patients = bundles.extract_entry(spark, resources, 'Patient')
encounter = bundles.extract_entry(spark, resources, 'Encounter')
condition = bundles.extract_entry(spark, resources, 'Condition')

In [ ]:
patients.printSchema()

In [ ]:
patients.select("id", "birthDate", "gender", "address.postalCode").toPandas()

In [ ]:
encounter.printSchema()

In [ ]:
encounter.select("id", "subject.reference", "serviceType.coding.code", "period.start", "period.end").toPandas()

In [ ]:
condition.printSchema()

In [ ]:
condition.select("id", "encounter.reference", "code.coding.code").toPandas()